In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import warnings
import chainladder as cl # Sparse=0.14.0 ; incremental.py np.NINF -> -np.inf

warnings.filterwarnings('ignore')

In [292]:
def claims_formatter(doc):
    doc_copy = doc.copy()
    doc_copy.index = doc_copy.AUTOMOTORES
    formatted_doc = doc_copy.iloc[:,4:].stack(level = [0,1,2,3]).reset_index([1,2,3,4]).drop_duplicates()
    formatted_doc.index = pd.MultiIndex.from_tuples(formatted_doc.index)

    formatted_doc.columns = ['Tipo', 'Periodo', 'Desarrollo 1', 'Año Valuacion', 'Pendientes', 'Pagados', 'Incurridos', 'Cantidad']
 
    formatted_doc['Año Ocurrencia'] = pd.to_datetime(formatted_doc['Periodo'][0][0:5]) + pd.DateOffset(months = 6)
    formatted_doc['Año Valuacion'] = pd.to_datetime(formatted_doc['Año Valuacion'])
    #formatted_doc['Desarrollo'] = formatted_doc['Año Valuacion'] - formatted_doc['Año Ocurrencia']
    formatted_doc.drop(['Tipo', 'Periodo', 'Desarrollo 1'], axis = 1, inplace = True)
    return formatted_doc


In [305]:
sto_input = pd.read_excel(r'D:\Documentos\Python_script\Github\IBNR-NIIF-17\sample\ssn_20192020_desarrollo_siniestros_automotores.xlsx', sheet_name = None, header=[0,1,2,3,4,5])
facpce_input = pd.read_excel(r'D:\Documentos\Python_script\Github\IBNR-NIIF-17\sample\Indice-FACPCE-Res.-JG-539-18-_2025-07-1.xlsx', skiprows = 2, skipfooter=4)

#Tratamiendo desarrollo de stos
sto_input = dict(itertools.islice(sto_input.items(), 3, len(sto_input)))
sto_input_formateado = [claims_formatter(sto_input[k]) for k in sto_input.keys()]
db_stos = pd.concat(sto_input_formateado)

#Tratamiento indice facpce
db_facpce = facpce_input.set_index('MES')
factores_facpce = db_facpce.loc['2024-06-01'] / db_facpce.loc[db_stos['Año Valuacion'] - pd.DateOffset(months = 5, days = 29)]
db_stos['Factor FACPSE'] = factores_facpce.set_index(db_stos.index)

In [306]:
db_stos

Año Valuacion    Pendientes  \
[HG]      Total                           NaN NaN    2019-06-30  2.515758e+10   
                                              NaN    2020-06-30  1.563698e+10   
          RC lesiones 3º transportados    1   NaN    2019-06-30  2.684507e+08   
                                              NaN    2020-06-30  5.230980e+08   
          RC lesiones 3º no transportados 2   NaN    2019-06-30  4.766707e+09   
...                                                         ...           ...   
[TV x HG] NaN                             M3  8.0    2003-06-30  0.000000e+00   
                                              8.0    2004-06-30  0.000000e+00   
                                              8.0    2005-06-30  0.000000e+00   
                                              8.0    2006-06-30  0.000000e+00   
                                              8.0    2007-06-30  0.000000e+00   

                                                        Pagados    Incurridos  \
[HG]      Total                           NaN NaN  4.274730e+10  6.790487e+10   
                                              NaN  6.353558e+10  7.917256e+10   
          RC lesiones 3º transportados    1   NaN  1.497806e+08  4.182314e+08   
                                              NaN  3.771734e+08  9.002714e+08   
          RC lesiones 3º no transportados 2   NaN  2.909627e+09  7.676334e+09   
...                                                         ...           ...   
[TV x HG] NaN                             M3  8.0  6.524000e+03  6.524000e+03   
                                              8.0  6.524000e+03  6.524000e+03   
                                              8.0  6.524000e+03  6.524000e+03   
                                              8.0  6.524000e+03  6.524000e+03   
                                              8.0  6.524000e+03  6.524000e+03   

                                                    Cantidad Año Ocurrencia  \
[HG]      Total                           NaN NaN  3310059.0     2018-07-01   
                                              NaN  3414254.0     2018-07-01   
          RC lesiones 3º transportados    1   NaN    20915.0     2018-07-01   
                                              NaN    22713.0     2018-07-01   
          RC lesiones 3º no transportados 2   NaN   173843.0     2018-07-01   
...                                                      ...            ...   
[TV x HG] NaN                             M3  8.0       56.0     1998-07-01   
                                              8.0       56.0     1998-07-01   
                                              8.0       56.0     1998-07-01   
                                              8.0       56.0     1998-07-01   
                                              8.0       56.0     1998-07-01   

                                                   Factor FACPSE  
[HG]      Total                           NaN NaN      33.498819  
                                              NaN      21.915318  
          RC lesiones 3º transportados    1   NaN      33.498819  
                                              NaN      21.915318  
          RC lesiones 3º no transportados 2   NaN      33.498819  
...                                                          ...  
[TV x HG] NaN                             M3  8.0     384.378212  
                                              8.0     379.875674  
                                              8.0     354.378198  
                                              8.0     312.981457  
                                              8.0     294.970105  

[46704 rows x 7 columns]

In [307]:
idx = pd.IndexSlice
test_triangle = db_stos.loc[idx['[HG]', 'RC', :, :], ['Año Ocurrencia', 'Año Valuacion', 'Pendientes','Pagados' , 'Incurridos', 'Factor FACPSE']]

In [308]:
test_triangle.sort_values(['Año Ocurrencia','Año Valuacion'])
aaaa = test_triangle.sort_values(['Año Ocurrencia','Año Valuacion'])['Pendientes'] - test_triangle.sort_values(['Año Ocurrencia','Año Valuacion'])['Pendientes'].shift(1)

In [309]:
aaa = test_triangle.reset_index(drop = True)

In [310]:
triang = cl.Triangle(test_triangle, origin = 'Año Ocurrencia', development= 'Año Valuacion', columns = 'Pagados', cumulative = True)

In [311]:
triang_2 = cl.Triangle(test_triangle, origin = 'Año Ocurrencia', development= 'Año Valuacion', columns = 'Factor FACPSE', cumulative = True)

In [312]:
triang

,12,24,36,48,60,72,84,96,108,120,...,156,168,180,192,204,216,228,240,252,264
1998,"146,588,786","255,722,322","319,456,874","365,073,290","407,447,109","459,143,148","497,094,746","521,711,463","547,664,803","569,408,138",...,"632,511,420","644,169,048","654,157,665","669,266,366","676,211,641","703,734,775","715,397,130","739,035,081","746,322,425","774,133,458"
1999,"153,371,174","285,807,201","345,485,013","391,535,441","448,845,518","491,139,685","533,330,165","565,689,813","584,500,368","619,818,308",...,"677,039,693","698,944,230","713,349,296","727,081,621","744,204,117","760,887,318","813,121,325","841,062,359","849,179,575",
2000,"172,984,640","329,612,364","390,332,924","451,909,761","507,237,511","553,850,246","587,170,407","627,690,935","669,926,966","700,023,188",...,"769,189,696","806,960,546","832,123,238","949,962,638","976,361,510","1,013,622,757","1,036,297,699","1,079,567,107",,
2001,"184,052,863","358,918,867","435,764,567","487,215,030","534,106,179","576,238,013","606,381,151","648,177,912","684,299,436","718,049,220",...,"818,479,108","868,703,773","890,713,846","912,487,162","941,245,005","962,545,316","985,690,183",,,
2002,"220,862,368","430,003,452","499,629,650","551,995,859","594,794,582","635,355,877","678,707,585","709,417,959","752,911,599","781,714,406",...,"876,174,847","902,081,270","927,534,046","980,233,126","1,035,201,123","1,120,870,919",,,,
2003,"265,221,772","528,816,885","631,072,274","700,244,619","748,363,322","801,919,402","857,873,769","912,257,108","961,393,241","1,010,999,442",...,"1,185,153,427","1,243,209,205","1,328,065,383","1,387,736,210","1,450,988,743",,,,,
2004,"368,876,819","734,812,738","863,285,611","940,786,706","1,014,687,244","1,090,722,040","1,166,480,816","1,229,251,648","1,287,244,667","1,346,260,447",...,"1,590,572,114","1,675,546,769","1,786,376,388","1,865,999,498",,,,,,
2005,"486,596,603","968,445,684","1,120,613,914","1,232,414,708","1,339,467,794","1,446,503,667","1,545,006,516","1,632,508,331","1,734,021,101","1,843,740,547",...,"2,270,224,260","2,414,725,560","2,531,913,532",,,,,,,
2006,"622,975,541","1,219,180,219","1,440,447,577","1,590,209,192","1,749,807,183","1,918,360,658","2,060,041,821","2,201,126,136","2,392,576,243","2,579,300,525",...,"3,174,734,733","3,365,160,200",,,,,,,,
2007,"809,240,549","1,587,797,265","1,873,643,702","2,092,718,603","2,284,250,200","2,472,101,073","2,675,638,530","2,898,408,106","3,220,762,986","3,512,163,025",...,"4,455,794,593",,,,,,,,,


In [313]:
triang_2

,12,24,36,48,60,72,84,96,108,120,...,156,168,180,192,204,216,228,240,252,264
1998,829.40,806.17,796.42,790.97,384.38,379.88,354.38,312.98,294.97,256.36,...,185.93,165.24,145.98,122.43,99.98,78.43,62.53,50.02,33.50,21.92
1999,806.17,796.42,790.97,384.38,379.88,354.38,312.98,294.97,256.36,237.64,...,165.24,145.98,122.43,99.98,78.43,62.53,50.02,33.50,21.92,
2000,796.42,790.97,384.38,379.88,354.38,312.98,294.97,256.36,237.64,212.47,...,145.98,122.43,99.98,78.43,62.53,50.02,33.50,21.92,,
2001,790.97,384.38,379.88,354.38,312.98,294.97,256.36,237.64,212.47,185.93,...,122.43,99.98,78.43,62.53,50.02,33.50,21.92,,,
2002,384.38,379.88,354.38,312.98,294.97,256.36,237.64,212.47,185.93,165.24,...,99.98,78.43,62.53,50.02,33.50,21.92,,,,
2003,379.88,354.38,312.98,294.97,256.36,237.64,212.47,185.93,165.24,145.98,...,78.43,62.53,50.02,33.50,21.92,,,,,
2004,354.38,312.98,294.97,256.36,237.64,212.47,185.93,165.24,145.98,122.43,...,62.53,50.02,33.50,21.92,,,,,,
2005,312.98,294.97,256.36,237.64,212.47,185.93,165.24,145.98,122.43,99.98,...,50.02,33.50,21.92,,,,,,,
2006,294.97,256.36,237.64,212.47,185.93,165.24,145.98,122.43,99.98,78.43,...,33.50,21.92,,,,,,,,
2007,256.36,237.64,212.47,185.93,165.24,145.98,122.43,99.98,78.43,62.53,...,21.92,,,,,,,,,


In [314]:
triang_act = triang.cum_to_incr() * triang_2

In [315]:
triang_act.incr_to_cum()

,12,24,36,48,60,72,84,96,108,120,...,156,168,180,192,204,216,228,240,252,264
1998,"121,581,316,637","209,561,379,013","260,320,982,390","296,402,114,259","312,689,687,010","332,327,754,844","345,776,973,896","353,481,549,766","361,137,009,074","366,711,156,948",...,"380,257,317,677","382,183,666,173","383,641,840,453","385,491,671,877","386,186,052,831","388,344,626,215","389,073,821,397","390,256,143,263","390,500,260,687","391,109,748,336"
1999,"123,643,069,822","229,118,044,439","276,321,292,982","294,022,074,242","315,792,778,126","330,780,908,946","343,985,746,642","353,530,875,581","358,353,172,244","366,746,143,069",...,"377,651,128,029","380,848,831,180","382,612,513,135","383,985,455,933","385,328,332,228","386,371,458,559","388,984,096,536","389,920,088,183","390,097,979,560",
2000,"137,768,784,905","261,656,325,934","284,995,986,096","308,387,528,447","327,994,476,753","342,583,398,547","352,411,850,121","362,799,742,021","372,836,730,985","379,231,321,571",...,"390,997,227,369","395,621,695,438","398,137,433,709","407,379,295,275","409,029,899,303","410,893,630,490","411,653,214,300","412,601,477,140",,
2001,"145,579,952,419","212,794,634,138","241,986,446,169","260,219,368,560","274,895,428,767","287,323,060,175","295,050,591,728","304,983,192,448","312,657,988,110","318,933,209,663",...,"333,620,302,353","338,641,709,063","340,367,906,347","341,729,294,932","343,167,703,326","343,881,238,593","344,388,465,718",,,
2002,"84,894,681,941","164,342,292,115","189,016,298,718","205,405,951,273","218,030,294,983","228,428,638,009","238,730,756,990","245,255,837,443","253,342,769,847","258,102,249,584",...,"269,702,884,273","271,734,662,839","273,326,111,764","275,962,011,720","277,803,374,734","279,680,855,563",,,,
2003,"100,751,299,208","194,163,660,489","226,167,701,040","246,571,475,007","258,907,244,272","271,634,334,726","283,523,045,043","293,634,739,354","301,754,171,307","308,995,862,866",...,"326,537,960,791","330,167,930,379","334,412,262,518","336,411,164,751","337,797,364,154",,,,,
2004,"130,721,902,205","245,253,059,369","283,148,716,337","303,016,991,428","320,578,747,651","336,733,977,734","350,820,085,653","361,192,564,506","369,658,593,842","376,884,181,756",...,"396,489,328,460","400,739,586,525","404,452,247,926","406,197,213,717",,,,,,
2005,"152,295,713,888","294,426,788,001","333,436,820,681","360,005,210,606","382,750,944,303","402,652,517,812","418,929,484,241","431,703,313,787","444,132,014,032","455,101,643,451",...,"481,266,660,342","486,107,283,328","488,675,495,022",,,,,,,
2006,"183,759,160,976","336,602,918,298","389,184,990,592","421,005,071,064","450,679,712,666","478,532,097,261","499,215,222,764","516,488,859,034","535,629,829,944","550,274,167,563",...,"578,754,700,967","582,927,935,682",,,,,,,,
2007,"207,457,892,616","392,474,453,357","453,208,685,015","493,942,087,300","525,591,459,793","553,014,605,538","577,934,682,484","600,206,939,454","625,488,459,560","643,708,407,915",...,"677,786,654,526",,,,,,,,,


In [316]:
triang_act.incr_to_cum().link_ratio.heatmap()

,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120,120-132,132-144,144-156,156-168,168-180,180-192,192-204,204-216,216-228,228-240,240-252,252-264
1998,1.7236,1.2422,1.1386,1.0550,1.0628,1.0405,1.0223,1.0217,1.0154,1.0167,1.0103,1.0095,1.0051,1.0038,1.0048,1.0018,1.0056,1.0019,1.0030,1.0006,1.0016
1999,1.8531,1.2060,1.0641,1.0740,1.0475,1.0399,1.0277,1.0136,1.0234,1.0131,1.0092,1.0071,1.0085,1.0046,1.0036,1.0035,1.0027,1.0068,1.0024,1.0005,
2000,1.8992,1.0892,1.0821,1.0636,1.0445,1.0287,1.0295,1.0277,1.0172,1.0158,1.0085,1.0064,1.0118,1.0064,1.0232,1.0041,1.0046,1.0018,1.0023,,
2001,1.4617,1.1372,1.0753,1.0564,1.0452,1.0269,1.0337,1.0252,1.0201,1.0190,1.0156,1.0107,1.0151,1.0051,1.0040,1.0042,1.0021,1.0015,,,
2002,1.9358,1.1501,1.0867,1.0615,1.0477,1.0451,1.0273,1.0330,1.0188,1.0147,1.0199,1.0096,1.0075,1.0059,1.0096,1.0067,1.0068,,,,
2003,1.9272,1.1648,1.0902,1.0500,1.0492,1.0438,1.0357,1.0277,1.0240,1.0226,1.0202,1.0130,1.0111,1.0129,1.0060,1.0041,,,,,
2004,1.8761,1.1545,1.0702,1.0580,1.0504,1.0418,1.0296,1.0234,1.0195,1.0218,1.0160,1.0134,1.0107,1.0093,1.0043,,,,,,
2005,1.9333,1.1325,1.0797,1.0632,1.0520,1.0404,1.0305,1.0288,1.0247,1.0181,1.0199,1.0184,1.0101,1.0053,,,,,,,
2006,1.8318,1.1562,1.0818,1.0705,1.0618,1.0432,1.0346,1.0371,1.0273,1.0216,1.0162,1.0131,1.0072,,,,,,,,
2007,1.8918,1.1547,1.0899,1.0641,1.0522,1.0451,1.0385,1.0421,1.0291,1.0262,1.0172,1.0087,,,,,,,,,
